In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-06-28 11:47:36--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-28 11:47:36 (57.9 MB/s) - ‘minsearch.py.1’ saved [4073/4073]



In [3]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

--2025-06-28 11:49:58--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  2.85MB/s    in 0.2s    

2025-06-28 11:49:59 (2.85 MB/s) - ‘documents.json’ saved [658332/658332]



In [5]:
import json
import minsearch

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
index.fit(documents)

In [7]:
q = 'the course has already started. Can I still join the course?'

In [8]:
boost = {'question': 3.0, 'section': 0.5}

result = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [9]:
result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [75]:
from mistralai import Mistral

In [76]:
client = Mistral(api_key="4JwtBsMx2DGAj0kfL2JpVn3j4f3vCkSI")

In [12]:
model = "mistral-large-latest"

In [13]:
client_response = client.chat.complete(
    model=model,
    messages=[
        {
            "role": "user",
            "content": q
        }
    ]
)

In [14]:
client_response.choices[0].message.content

"Whether or not you can still join a course that has already started depends on several factors, including the specific policies of the institution or organization offering the course, the nature of the course, and your personal circumstances. Here are some general considerations:\n\n1. **Institutional Policies**:\n   - **Universities and Colleges**: Many universities and colleges have add/drop periods at the beginning of the semester during which students can join or leave courses. After this period, it may be more difficult to join.\n   - **Online Courses and MOOCs**: Online courses and Massive Open Online Courses (MOOCs) often have more flexible enrollment policies. You might be able to join at any time, but you may need to catch up on missed material.\n\n2. **Nature of the Course**:\n   - **Content and Structure**: If the course is highly sequential, where each lesson builds on the previous one, joining late might be challenging. Conversely, if the course is modular or self-paced, 

In [25]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT dosen't contain the answer, output None.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [26]:
context = ""

for doc in result:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    

In [27]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: 

In [29]:
prompt = prompt_template.format(question=q, context=context).strip()

In [31]:
response = client.chat.complete(
    model=model,
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

In [34]:
response.choices[0].message.content

"Yes, you can still join the course after it has started. Even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects."

In [66]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    result = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=15
    )
    return result

In [69]:
def build_prompt(query, search_result):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT dosen't contain the answer, output None.
        
        QUESTION: {question}
        
        CONTEXT: 
        {context}
    """.strip()

    context = ""

    for doc in search_result:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [77]:
def llm(prompt):
    response = client.chat.complete(
    model="mistral-large-latest",
        messages=[{
            "role": "user",
            "content": prompt
    }]
    )
    return response.choices[0].message.content

In [65]:
def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_query=search_result)
    result = llm(prompt)
    return result 

In [76]:
print(rag("how do I run kafka?"))

To run Kafka, you can follow these instructions based on the provided context:

For Java Kafka, you can run the producer in the terminal using the following command in the project directory:
```
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

For Python Kafka, if you encounter issues with modules not being found, create a virtual environment and install the necessary packages:
```
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
```

Ensure that all Docker images are up and running before executing your Python files.


In [77]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
from elasticsearch import Elasticsearch

In [10]:
es_client = Elasticsearch('http://localhost:9200')

In [11]:
es_client.info()

ObjectApiResponse({'name': 'b7847b80d241', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'oGK9ystPRbC0VlkHbI67mQ', 'version': {'number': '8.13.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09df99393193b2c53d92899662a8b8b3c55b45cd', 'build_date': '2024-03-22T03:35:46.757803203Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [12]:
import elasticsearch
elasticsearch.__version__

(8, 13, 0)

In [42]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    }
}

index_name = "course-question"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-question'})

In [43]:
from tqdm.auto import tqdm

In [44]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 336.26it/s]


In [57]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
    

In [58]:
response = es_client.search(index=index_name, body=search_query)

In [59]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [60]:
result_docs

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repositor

In [85]:
def elastic_search(query):
    search_query = {
    "size": 20,
    "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs
    

In [89]:
elastic_search('How do I run kafka')

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repositor

In [93]:
def rag(query):
    search_result = elastic_search(query)
    prompt = build_prompt(query, search_result=search_result)
    result = llm(prompt)
    return result 
rag('how do i run kafka')

'To run Kafka, you need to follow the instructions specific to the type of Kafka implementation you are using. Based on the context provided:\n\nFor Java Kafka, you can run a producer in the terminal by navigating to the project directory and executing the following command:\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor running a Python Kafka producer, you should create a virtual environment and install the necessary dependencies. Here are the steps:\n\n1. Create a virtual environment:\n   ```sh\n   python -m venv env\n   ```\n\n2. Activate the virtual environment:\n   - On MacOS and Linux:\n     ```sh\n     source env/bin/activate\n     ```\n   - On Windows:\n     ```sh\n     env\\Scripts\\activate\n     ```\n\n3. Install the required packages:\n   ```sh\n   pip install -r ../requirements.txt\n   ```\n\n4. Ensure that Docker images are up and running before executing the Python file.\n\nIf you need to run Spark Stre